In [9]:
!pip install pytesseract

In [10]:
# PYTHON IMPORTS
import os, copy, math, re
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image

# DATA IMPORTS 
import random, h5py, glob
import numpy as np
import requests
import pandas as pd

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# SHAPES IMPORTS
import shapely
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import LineString, Polygon, Point

# MY OWN CLASSES
from FindGrid import *

# OCR libraries
from fuzzywuzzy import fuzz, process

# PREFERENCES
Image.MAX_IMAGE_PIXELS = 933120000

Download Places Datasets

In [11]:
if False:
    # Base URL pattern with a placeholder for the integer
    base_url = "https://www2.census.gov/geo/tiger/TIGER2022/PLACE/tl_2022_{:02d}_place.zip"  # Replace with your base URL
    savedir  = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\ReferenceDatasets\Places"
    # Loop through integers from 1 to 100
    for i in range(1, 101):
        # Construct the URL by formatting the integer into the base URL
        url = base_url.format(i)

        try:
            # Send an HTTP GET request to the URL
            response = requests.get(url)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Save the response content to a file with a filename based on the integer
                filename = f"{i}.zip"  # You can change the filename format
                with open(f"{savedir}/{filename}", "wb") as file:
                    file.write(response.content)
                print(f"Saved {filename}")
            else:
                print(f"Failed to retrieve {url}. Status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred while processing {url}: {str(e)}")
            
def strip_non_numeric_and_convert(field_value):
    return pd.to_numeric(field_value.str.replace(r'[^0-9]', ''))

In [12]:
data_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data/"

CIDs        = pd.read_csv(f"{data_dir}ReferenceDatasets/CommunityNumbers.csv")
CIDs["CID"] = strip_non_numeric_and_convert(CIDs["CID"])

In [15]:
CIDs

,CID,Community Name,County,Init FHBM Identified,Init FIRM Identified,Curr Eff Map Date,Reg-Emer Date,Tribal,CRS Entry Date,Curr Eff Date,Curr Class,% Disc SFHA,% Disc Non SFHA,Program,Participating Community
0,10259.0,"ABBEVILLE, CITY OF",HENRY COUNTY,"=""01/31/75""","=""09/04/85""","=""09/28/07""","=""09/04/85""",No,NaN,NaN,NaN,NaN,NaN,Regular,YES
1,10267.0,"ADAMSVILLE, CITY OF",JEFFERSON COUNTY,"=""01/17/75""",10/10/80,"=""03/21/19""",10/10/80,No,NaN,NaN,NaN,NaN,NaN,Regular,YES
2,10504.0,"ADDISON, TOWN OF",WINSTON COUNTY,NaN,"=""08/18/09""","=""09/16/11(M)""","=""04/01/13""",No,NaN,NaN,NaN,NaN,NaN,Regular,YES
3,10192.0,"ALABASTER, CITY OF",SHELBY COUNTY,"=""05/24/74""","=""06/15/81""","=""02/20/13""","=""06/15/81""",No,NaN,NaN,NaN,NaN,NaN,Regular,YES
4,10366.0,"ALBERTVILLE, CITY OF",MARSHALL COUNTY,10/01/76,"=""09/04/85""","=""03/21/19(M)""","=""09/04/85""",No,NaN,NaN,NaN,NaN,NaN,Regular,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25507,560003.0,"ROCK RIVER, CITY OF",ALBANY COUNTY,"=""02/07/75""","=""06/16/11""","=""06/16/11""","=""02/07/76""",No,NaN,NaN,NaN,NaN,NaN,NaN,NO
25508,560087.0,SWEETWATER COUNTY *,SWEETWATER COUNTY,"=""08/01/78""",NaN,"=""08/01/78""","=""08/01/79""",No,NaN,NaN,NaN,NaN,NaN,NaN,NO
25509,560109.0,"THAYNE, TOWN OF",LINCOLN COUNTY,NaN,11/16/11,11/16/11,11/16/12,No,NaN,NaN,NaN,NaN,NaN,NaN,NO
25510,560079.0,"UPTON, TOWN OF",WESTON COUNTY,"=""06/25/76""",NaN,"=""06/25/76""","=""06/25/77""",No,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [ ]:
# Function to find the best fuzzy match
def find_best_match(left_value, choices):
    return process.extractOne(left_value, choices, scorer=fuzz.ratio)

# Apply the fuzzy matching and merge the DataFrames
merged_data = []

for left_row in left_df['left_column']:
    best_match, _ = find_best_match(left_row, right_df['right_column'])
    merged_data.append((left_row, best_match))

merged_df = pd.DataFrame(merged_data, columns=['left_column', 'best_match'])

# Merge the DataFrames
result_df = pd.merge(left_df, merged_df, left_on='left_column', right_on='left_column', how='left')

print(result_df)
